<a href="https://colab.research.google.com/github/henry-heppe/SeminarML/blob/main/boatsProcessingInclNAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Initialize DataFrame

In [ ]:
import pandas as pd
from datetime import datetime
import numpy as np
import geocoder
import ast

In [ ]:
currentYear = datetime.now().year

In [ ]:
boatsDF = pd.read_csv('boatsData.csv')

<ipython-input-126-863b05290d90>:1: DtypeWarning: Columns (126,127,130,131,139,180,181,182) have mixed types. Specify dtype option on import or set low_memory=False.
  boatsDF = pd.read_csv('boatsData.csv')


In [ ]:
print(boatsDF.head())

       ID Original price                            TAXES PAID  \
0  Boat_1      EUR 9,990                         EU taxes paid   
1  Boat_2     EUR 28,590                 Basis for Negotiation   
2  Boat_3     EUR 65,000  Basis for Negotiation, EU taxes paid   
3  Boat_4    EUR 225,000  Basis for Negotiation, EU taxes paid   
4  Boat_5     EUR 72,500                                   NaN   

            Price  Year Built            Condition     Length x Beam  \
0    apx. £ 8,560      1961.0       good condition   7.75 m x 2.50 m   
1   apx. £ 24,500      2016.0       good condition   4.80 m x 2.15 m   
2   apx. £ 55,700      1997.0  very good condition  11.85 m x 3.85 m   
3  apx. £ 192,600      2010.0       good condition  12.00 m x 3.99 m   
4   apx. £ 62,100      2012.0       good condition   9.27 m x 3.10 m   

         Draught Displacement CE Design Category  ...  \
0  1.50 - 0.35 m     1'100 kg        C - Inshore  ...   
1         0.30 m       725 kg        C - Inshore  ...   

In [ ]:
print(boatsDF.shape[0])
print(boatsDF.shape[1])

27393
1188


# Drop Duplicates

In [ ]:
boatsDF = boatsDF.drop_duplicates(subset=['Year Built', 'Price', 'Length x Beam', 'Draught'], keep='first')
boatsDF = boatsDF.sort_values(by=['Year Built', 'Price', 'Length x Beam', 'Draught'])

# Remove Incomplete Samples

In [ ]:
# Remove Samples without a Price
boatsDF = boatsDF.dropna(subset='Price')

# Remove Non Boats

In [ ]:
# Make the string representation of the boat categories a list
boatsDF['Categories'] = boatsDF['Categories'].apply(ast.literal_eval)

# Get a set of all possible categories
categoriesList = boatsDF['Categories'].tolist()
uniqueCategories = list(set(value for sublist in categoriesList for value in sublist))
print(uniqueCategories)

# Non-Boat categories
equipmentCategories = [' Mooring Line', ' Buoy', 'Single Axle', 'Fittings', 'Port Trailer', ' Outdoor Storage', ' Floating Dock', 'Boat Engine', 'Engine Accessories', ' Storage Space',
                       'Storage Trailer', 'On-Board Electricity', 'Safety', 'Spare Part', 'Equipment', 'Twin Axle', 'Trailer', 'Triple Axle', ' Water Ski', ' Wakeboard/Wakesurf',
                       ' Fixed Dock', 'Berth']
boatsDF = boatsDF[~boatsDF['Categories'].apply(lambda categories: any(category in equipmentCategories for category in categories))]

# Commercial Vessels and Houses
nonPriceableBoats = [' Fishing Boat', ' Trawler', ' Boathouse', ' Floating House', ' Houseboat']
boatsDF = boatsDF[~boatsDF['Categories'].apply(lambda categories: any(category in nonPriceableBoats for category in categories))]

# Inflatable Boats and Miscellaneous
miscellaneousBoats = [' Pedalo', 'Inflatable Boat' , ' Foldable Inflatable Boat', ' Rowing Boat', 'Display Model', ' Canoe']
boatsDF = boatsDF[~boatsDF['Categories'].apply(lambda categories: any(category in miscellaneousBoats for category in categories))]

[' Runabout', ' Storage Space', ' Classic Power Boat', ' Pedalo', ' Rowing Boat', ' Centre cockpit', ' Offshore Boat', ' Boathouse', ' Sloep', ' Center Console', 'Inflatable Boat', ' Sailing Superyacht', ' Superyacht', 'Used Boat', ' Port Trailer', 'Trailer', ' Twin Axle', ' Motor Yacht', ' Water Ski', ' Dolphins', ' Classic Sailing Yacht', ' Keelboat', 'Equipment', ' Sails', ' On-Board Electricity', ' Storage Trailer', ' Fixed Dock', 'Boat Engine', ' Safety', ' Utility Boat', ' Pontoon Boat', ' Outdoor Storage', ' Flybridge', ' Daysailer', ' Spare Part', ' Canoe', ' Floating Dock', ' Passenger Ship', 'Berth', ' Bowrider', ' Wakeboard/Wakesurf', ' Regatta Boat', ' Mooring Line', ' Pilothouse', 'New Boat on Order', ' Foldable Inflatable Boat', ' Outboard', ' Sport Boat', ' Inboard', ' Buoy', ' Other', ' Sailing Dinghy', ' Houseboat', ' Single Axle', ' Motorsailer', ' Cabin Cruiser', 'Power Boat', ' RIB', ' Personal Water Craft', ' Deck Boat', 'Small Boat', ' Hardtop', ' Flat Bottom Boat

In [ ]:
categoriesList = boatsDF['Categories'].tolist()
uniqueCategories = list(set(value for sublist in categoriesList for value in sublist))
print(uniqueCategories)

[' Runabout', ' Sailing Yacht', ' Sport Boat', ' Classic Power Boat', ' Utility Boat', ' Gulet', ' Pontoon Boat', ' Centre cockpit', ' Flybridge', ' Offshore Boat', ' Sailing Dinghy', ' Sloep', ' Cruising Dinghy', ' Ketch', ' Motorsailer', ' Cabin Cruiser', ' Daysailer', ' Dinghy', 'Power Boat', ' Cruising Yacht', ' Center Console', ' Sailing Superyacht', ' Superyacht', 'Used Boat', ' RIB', ' Pilothouse', ' Passenger Ship', ' Motor Yacht', ' Trimaran', ' Personal Water Craft', ' Deck Boat', 'Small Boat', ' Hardtop', ' Flat Bottom Boat', ' Bowrider', ' Classic Sailing Yacht', ' Regatta Boat', 'New boat preordered', ' Power Catamaran', ' Deck Saloon', ' Keelboat', 'New Boat from Stock', ' Catamaran', 'New Boat on Order']


# Drop Variables

In [ ]:
#Get Columns
columnNames = boatsDF.columns

#Remove Columns with less than 200 non NaN values (Approx. 1 in 100 boats need to have the feature)
for column in columnNames:
    numberValidObservations = boatsDF[column].notna().sum()
    if numberValidObservations < 200:
      boatsDF = boatsDF.drop(column, axis=1)

#Remove Original Price column
boatsDF = boatsDF.drop('Original price', axis=1)

#Get Columns
columnNames = boatsDF.columns

# Rename Columns

In [ ]:
boatsDF = boatsDF.rename(columns={'TAXES PAID': 'Tax Description'})
boatsDF = boatsDF.rename(columns={'Year Built': 'Age'})

# Column Cleaning



In [ ]:
boatsDF['Price'] = boatsDF['Price'].str.replace('\D', '', regex=True).astype(int)
boatsDF['Age'] = boatsDF['Age'].astype('Int64')
boatsDF['Age'] = currentYear - boatsDF['Age']
boatsDF.loc[boatsDF['Age'] == 2024, 'Age'] = np.nan

# Column Mapping

In [ ]:
#Make a linear mapping for the condition column
conditionMap = {
    'damaged': 1,
    'in need of repair': 2,
    'used': 3,
    'fair condition': 4,
    'good condition': 5,
    'very good condition': 6,
    'as new': 7,
    'new': 8
}

boatsDF['Condition'] = boatsDF['Condition'].str.strip().str.lower()
boatsDF['Condition'] = boatsDF['Condition'].map(conditionMap)
boatsDF['Condition'] = boatsDF['Condition'].astype('Int64')

In [ ]:
#Separate Length and Beam
#Make Draught a float
#Make Displacement an int

boatsDF['Length'] = boatsDF['Length x Beam'].str.extract(r'(\d+\.\d+)', expand=True)
boatsDF['Beam'] = boatsDF['Length x Beam'].str.extract(r'.*?\d+\.\d+\D+(\d+\.\d+)', expand=True)
boatsDF['Draught'] = boatsDF['Draught'].str.extract(r'(\d+\.\d+)')
boatsDF['Displacement'] = boatsDF['Displacement'].str.replace("'", "").str.extract('(\d+)')

boatsDF['Length'] = boatsDF['Length'].astype(float)
boatsDF['Beam'] = boatsDF['Beam'].astype(float)
boatsDF['Draught'] = boatsDF['Draught'].astype(float)
boatsDF['Displacement'] = boatsDF['Displacement'].astype('Int64')

#Drop the Length x Beam column
boatsDF.drop('Length x Beam', axis=1, inplace=True)

#Drop the Views, favourites, ID and listings column
boatsDF.drop('Views, favourites, ID and listings', axis=1, inplace=True)

#Drop the URL column
boatsDF.drop('URL', axis=1, inplace=True)

In [ ]:
#Make two new columns based on the Tax Description column

# Function to check if it possible to negotiate on the price
def check_basis_for_negotiation(text):
    if pd.isnull(text):
        return 0
    return 1 if 'Basis for Negotiation' in text else 0

# Function to check if taxes are paid
def check_taxes_paid(text):
    if pd.isnull(text):
        return 0
    phrases_indicating_taxes_paid = ['incl. VAT', 'EU taxes paid', 'incl. 8.1% Swiss VAT']
    return 1 if any(phrase in text for phrase in phrases_indicating_taxes_paid) else 0

# Apply these functions to create new columns
boatsDF['Negotiation Possible'] = boatsDF['Tax Description'].apply(check_basis_for_negotiation)
boatsDF['Taxes Paid'] = boatsDF['Tax Description'].apply(check_taxes_paid)

# Drop the Tax Description column
boatsDF.drop('Tax Description', axis=1, inplace=True)

In [ ]:
# Create dummy variables for CE Design Category
designCategoryDummies = pd.get_dummies(boatsDF['CE Design Category'], prefix='CE Design Category', dummy_na=True)
boatsDF = pd.concat([boatsDF, designCategoryDummies], axis=1)

# Drop the CE Design Category column
boatsDF.drop('CE Design Category', axis=1, inplace=True)

In [ ]:
# Create dummy variables for Hull Type
hullTypeDummies = pd.get_dummies(boatsDF['Hull Type'], prefix='Hull Type', dummy_na=True)
boatsDF = pd.concat([boatsDF, hullTypeDummies], axis=1)

# Drop the Hull Type column
boatsDF.drop('Hull Type', axis=1, inplace=True)

In [ ]:
# Create dummy variables for Material
materialDummies = pd.get_dummies(boatsDF['Material'], prefix='Material', dummy_na=True)
boatsDF = pd.concat([boatsDF, materialDummies], axis=1)

# Drop the Hull Type column
boatsDF.drop('Material', axis=1, inplace=True)

In [ ]:
# Clean Certified No. of Persons column
boatsDF['Certified No. of Persons'] = boatsDF['Certified No. of Persons'].str.extract('(\d+)').astype('Int64')

# Clean Headroom column
boatsDF['Headroom'] = boatsDF['Headroom'].str.extract('(\d+)').astype('Int64')

# Clean No. of Cabins column
boatsDF['No. of Cabins'] = boatsDF['No. of Cabins'].str.extract('(\d+)').astype('Int64')

# Clean No. of Bathrooms column
boatsDF['No. of Bathrooms'] = boatsDF['No. of Bathrooms'].str.extract('(\d+)').astype('Int64')

# Clean No. of Berths column
boatsDF['No. of berths'] = boatsDF['No. of berths'].str.extract('(\d+)').astype('Int64')

In [ ]:
# Drop the Fresh water tank column
boatsDF.drop('Fresh water tank', axis=1, inplace=True)

# Drop the Holding Tank column
boatsDF.drop('Holding Tank', axis=1, inplace=True)

# Drop the Propulsion column
boatsDF.drop('Propulsion', axis=1, inplace=True)

# Drop the Engine column
boatsDF.drop('Engine', axis=1, inplace=True)

# Drop the VAT column
boatsDF.drop('VAT', axis=1, inplace=True)

# Drop the Boat Name column
boatsDF.drop('Boat Name', axis=1, inplace=True)

# Drop the Weight column (remember, we have Displacement)
boatsDF.drop('Weight', axis=1, inplace=True)

# Drop the Cruising Speed, Max Speed, Ballast and Engine Year column (too many NaN values)
boatsDF.drop('Cruising Speed', axis=1, inplace=True)
boatsDF.drop('Max speed', axis=1, inplace=True)
boatsDF.drop('Ballast', axis=1, inplace=True)
boatsDF.drop('Engine Year', axis=1, inplace=True)

# Drop the Hull Color column
boatsDF.drop('Hull Color', axis=1, inplace=True)

# Drop the iPod interface column
boatsDF.drop('iPod Interface', axis=1, inplace=True)

In [ ]:
# Distinguish between Sailing Vessels and Motorized Vessels
boatCategoryMap = {
    'Sailing Vessel': ['Ketch', 'Catamaran', 'Sailing Dinghy', 'Daysailer', 'Sailing Yacht',  'Classic Sailing Yacht', 'Sailing Superyacht',
                        'Regatta Boat', 'Trimaran', 'Motorsailer', 'Gulet', 'Dinghy', 'Cruising Dinghy', 'Keelboat', 'Sloep'],
    'Motorized Vessel': ['Superyacht', 'Pilothouse', 'Flybridge', 'Deck Boat', 'Passenger Ship', 'Motor Yacht', 'Power Catamaran',
                          'Classic Power Boat', 'Offshore Boat', 'Utility Boat', 'Power Boat', 'Runabout', 'Bowrider', 'Personal Water Craft',
                          'RIB', 'Center Console', 'Cabin Cruiser', 'Cruising Yacht', 'Deck Saloon', 'Pontoon Boat', 'Sport Boat', 'Flat Bottom Boat',
                          'Hardtop', 'Centre Cockpit']
}

# Function to make the dummy columns
def addCategoryColumns(df, boatCategoryMap):
    for categoryName, categoryList in boatCategoryMap.items():
        # Trim whitespaces and adjust for case sensitivity in comparison
        df[categoryName] = df['Categories'].apply(
            lambda x: 0 if x is None else 1 if any(item.strip().lower() in [cat.lower() for cat in categoryList] for item in x) else 0
        )
    return df

boatsDF = addCategoryColumns(boatsDF, boatCategoryMap)

# Drop the Categories column
boatsDF.drop('Categories', axis=1, inplace=True)

# print((boatsDF['Sailing Vessel'] == 1).sum())
# print((boatsDF['Motorized Vessel'] == 1).sum())
# print(boatsDF[['Sailing Vessel', 'Motorized Vessel']].isna().all(axis=1).sum())

In [ ]:
#Make 1-0 dummy columns for the equipment columns and make them int type
columnsToUpdate = [
    "Shore Connection", "Battery", "Battery Charger", "Fire Extinguisher", "Anchor",
    "Bow Anchor Capstan", "Bilge Pump", "Navigation Lights", "Bow Thruster", "Electric Toilet",
    "Water Pressure System", "Warm Water System", "Board Computer", "Compass", "GPS",
    "Autopilot", "VHF Radio", "Chartplotter", "Depth Instrument", "Speed Instrument",
    "Stove", "Sink", "Fridge", "Gas Stove", "Full Cover", "Cockpit Cover", "Bimini Top",
    "Swim Ladder", "Bathing plaftorm fix", "Deck Shower", "Heating", "Radio", "CD Player",
    "Cockpit Speakers", "Cockpit Table", "Teak Cockpit", "Underwater Paint", "Inverter", "Bluetooth Connector",
    'Radar Reflector', 'Trailer', 'Generator', 'Electric winch', 'Trim Tabs', 'Fishfinder', 'Radar', 'Cool Box',
    'DVB-T Antenna', 'Wetbar', 'Teak Deck', 'No. of Bathrooms', 'Alarm system', 'Fire extinguisher system', 'Watermaker',
    'Deck washer', 'Solar panels', 'Ceran Stove', 'Microwave', 'Oven', 'Freezer', 'Electric Stove', 'Extractor Hood', 'Dishwasher',
    'Air Conditioning', 'MP3 Player', 'BBQ-Grill', 'Gas Detector', 'EPIRB', 'Flybridge Cover', 'Winter Cover', 'Gangway', 'TV',
    'Satellite Antenna', 'Stern Anchor Capstan', 'Life Raft', 'Washing Machine', 'AIS', 'MOB System', 'Icemaker', 'DVD Player',
    'Underwater Lights', 'Windvane self-steering', 'Bathing platform electric', 'Stern Thruster', 'Electric Gangway', 'Davits',
    'Wind Instrument', 'Sprayhood', 'Bathing platform manual']

boatsDF[columnsToUpdate] = boatsDF[columnsToUpdate].fillna(0).astype(int)

In [ ]:
# Make a Small Equipment Index column
smallEquipmentColumns = ["Fire Extinguisher", "Anchor", "Navigation Lights", "Compass", "Depth Instrument", "Speed Instrument", "Full Cover", "Cockpit Cover", "Bimini Top", "Swim Ladder", "Deck Shower",
                         "Radio", "CD Player", "Cockpit Speakers", "Cockpit Table", "Underwater Paint", "Bluetooth Connector", 'Radar Reflector', 'Cool Box', 'DVB-T Antenna', 'MP3 Player', 'BBQ-Grill',
                         'Gas Detector', 'EPIRB', 'AIS', 'MOB System', 'DVD Player', 'Wind Instrument', 'Sprayhood']
boatsDF['Small Equipment Index'] = boatsDF[smallEquipmentColumns].sum(axis=1)

# Drop the original Small Equipment columns
boatsDF.drop(smallEquipmentColumns, axis=1, inplace=True)

# Make a Big Equipment Index column
bigEquipmentColumns = ["Shore Connection", "Battery", "Battery Charger", "Bow Anchor Capstan", "Bow Thruster", "Electric Toilet", "Water Pressure System", "Warm Water System", "Board Computer", "GPS", "Autopilot", "VHF Radio",
                       "Chartplotter", "Fridge", "Bilge Pump", "Stove", "Sink", "Gas Stove", "Bathing plaftorm fix", "Heating", "Teak Cockpit", "Inverter", 'Trailer', 'Generator', 'Electric winch', 'Trim Tabs', 'Fishfinder',
                       'Radar', 'Wetbar', 'Teak Deck', 'Alarm system', 'Fire extinguisher system', 'Watermaker', 'Deck washer', 'Solar panels', 'Ceran Stove', 'Microwave', 'Oven', 'Freezer', 'Electric Stove', 'Extractor Hood',
                       'Dishwasher', 'Air Conditioning', 'Flybridge Cover', 'Winter Cover', 'Gangway', 'TV', 'Satellite Antenna', 'Stern Anchor Capstan', 'Life Raft', 'Washing Machine', 'Icemaker', 'Underwater Lights',
                       'Windvane self-steering', 'Bathing platform electric', 'Stern Thruster', 'Electric Gangway', 'Davits', 'Bathing platform manual']
boatsDF['Big Equipment Index'] = boatsDF[bigEquipmentColumns].sum(axis=1)

# Drop the original Big Equipment columns
boatsDF.drop(bigEquipmentColumns, axis=1, inplace=True)

# Make a column that is equal to 1 if both equipment columns are 0 and 0 otherwise
boatsDF['Equipment Not Listed'] = ((boatsDF['Small Equipment Index'] + boatsDF['Big Equipment Index']) == 0).astype(int)

<ipython-input-146-cdfb56d758a7>:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  boatsDF['Small Equipment Index'] = boatsDF[smallEquipmentColumns].sum(axis=1)


In [ ]:
# Clean Engine Performance column
boatsDF['Horsepower'] = boatsDF['Engine Performance'].str.extract(r'(\d+\.?\d*)\s*x?\s*HP')[0].astype(float)
boatsDF['Engine Count'] = boatsDF['Engine Performance'].str.extract(r'(\d+)\s*x')[0].fillna(1).astype(int)
boatsDF['Total Horsepower'] = boatsDF['Horsepower'] * boatsDF['Engine Count']

# Drop the Engine Performance, Horsepower and Engine Count columns
boatsDF.drop('Engine Performance', axis=1, inplace=True)
boatsDF.drop('Horsepower', axis=1, inplace=True)
boatsDF.drop('Engine Count', axis=1, inplace=True)

In [ ]:
# Make a Fuel Capacity column
boatsDF['Fuel Capacity'] = boatsDF['Fuel Type'].str.extract(r"(\d{1,3}(?:'\d{3})*|\d+)\s*l")[0].str.replace("'", "").astype('Int64')

# Drop the Fuel Type column
boatsDF.drop('Fuel Type', axis=1, inplace=True)

In [ ]:
# Clean the Engine Hours column
boatsDF['Engine Hours'] = boatsDF['Engine Hours'].str.replace("'", "").str.extract(r'(\d+)')[0].astype('Int64')

In [ ]:
# Make a Country column
boatsDF['Country'] = boatsDF['Location'].str.extract(r'^(.+?)(?:\s»|$)')

# Drop the Location column
boatsDF.drop('Location', axis=1, inplace=True)

In [ ]:
# Make a Sail Area columns
boatsDF['Mainsail Area'] = boatsDF['Mainsail'].str.replace("'", "").str.extract(r'(\d+\.?\d*)')[0].astype('Int64')
boatsDF['Jib Area'] = boatsDF['Jib'].str.replace("'", "").str.extract(r'(\d+\.?\d*)')[0].astype('Int64')
boatsDF['Genoa Area'] = boatsDF['Genoa'].str.replace("'", "").str.extract(r'(\d+\.?\d*)')[0].astype('Int64')
boatsDF['Spinnaker Area'] = boatsDF['Spinnaker'].str.replace("'", "").str.extract(r'(\d+\.?\d*)')[0].astype('Int64')
boatsDF['Gennaker Area'] = boatsDF['Gennaker'].str.replace("'", "").str.extract(r'(\d+\.?\d*)')[0].astype('Int64')

# Drop the Sail columns
boatsDF.drop('Mainsail', axis=1, inplace=True)
boatsDF.drop('Jib', axis=1, inplace=True)
boatsDF.drop('Genoa', axis=1, inplace=True)
boatsDF.drop('Spinnaker', axis=1, inplace=True)
boatsDF.drop('Gennaker', axis=1, inplace=True)

In [ ]:
# Clean the Showers column
boatsDF['Showers'] = boatsDF['Showers'].str.extract('(\d+)').astype('Int64')

# Clean the Toilets column
boatsDF['Toilets'] = boatsDF['Toilets'].str.extract('(\d+)').astype('Int64')

In [ ]:
# Create dummy variables for Steering
steeringDummies = pd.get_dummies(boatsDF['Steering'], prefix='Steering', dummy_na=True)
boatsDF = pd.concat([boatsDF, steeringDummies], axis=1)

# Drop the Steering column
boatsDF.drop('Steering', axis=1, inplace=True)

In [ ]:
# Create dummy variables for Keel Type
keelTypeDummies = pd.get_dummies(boatsDF['Keel Type'], prefix='Keel Type', dummy_na=True)
boatsDF = pd.concat([boatsDF, keelTypeDummies], axis=1)

# Drop the Keel Type column
boatsDF.drop('Keel Type', axis=1, inplace=True)

In [ ]:
# Drop the Description column
boatsDF.drop('Description', axis=1, inplace=True)

In [ ]:
# Drop the Key and ID column
boatsDF.drop('ID', axis=1, inplace=True)
boatsDF.drop('Key', axis=1, inplace=True)

# Removing Bad Outliers


In [ ]:
# Set Displacement larger than 50 mil equal to NaN
boatsDF.loc[boatsDF['Displacement'] > 50000000, 'Displacement'] = np.nan

# Set No. of berths larger than 20 equal to NaN
boatsDF.loc[boatsDF['No. of berths'] > 20, 'No. of berths'] = np.nan

# Set Headroom larger than 300 equal to NaN
boatsDF.loc[boatsDF['Headroom'] > 300, 'Headroom'] = np.nan

# Set Certified No. of Persons larger than 30 equal to NaN
boatsDF.loc[boatsDF['Certified No. of Persons'] > 30, 'Certified No. of Persons'] = np.nan

# Set Draught larger than 10 equal to NaN
boatsDF.loc[boatsDF['Draught'] > 10, 'Draught'] = np.nan

# Smart Data Decisions

In [ ]:
boatsDF = boatsDF[boatsDF['Price'] <= 1000000].copy()

# Make a Ship Area column
boatsDF['Ship Area'] = boatsDF['Length'] * boatsDF['Beam']

# Drop samples for which Ship Area is not defined
boatsDF = boatsDF.loc[boatsDF['Ship Area'].notna()]

# Drop samples for which Age is not defined
boatsDF = boatsDF.loc[boatsDF['Age'].notna()]

# Merge With Images

In [ ]:
imagesDF = pd.read_csv('image_features.csv', sep = ';')

In [ ]:
boatsDF = boatsDF[boatsDF['Price'] <= 500000]

In [ ]:
boatsDF = boatsDF.reset_index(drop=True)
boatsDF = pd.concat([boatsDF, imagesDF], axis=1)

# Remove Bad Location Row

In [ ]:
boatsDF = boatsDF[~(boatsDF['Country'] == 'Channel Islands, British Isles')]
boatsDF['Country'][boatsDF['Country'] == 'Contact De Valk Loosdrecht'] = 'Netherlands'

<ipython-input-163-26e78f0d3129>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  boatsDF['Country'][boatsDF['Country'] == 'Contact De Valk Loosdrecht'] = 'Netherlands'


# Get Coordinates

In [ ]:
countries = boatsDF['Country']
countrySet = set(countries)
geoDictionary = {key: key for key in countrySet}
for country in countrySet:
  geoCode = geocoder.osm(country)
  if geoCode.ok:
    geoDictionary[country] = {'latitude': geoCode.lat, 'longitude': geoCode.lng}
  else:
    geoDictionary[country] = 'Geocode lookup failed'

In [ ]:
def get_latitude(country):
    return geoDictionary[country]['latitude'] if country in geoDictionary and 'latitude' in geoDictionary[country] else None

def get_longitude(country):
    return geoDictionary[country]['longitude'] if country in geoDictionary and 'longitude' in geoDictionary[country] else None

boatsDF['Latitude'] = boatsDF['Country'].apply(get_latitude)
boatsDF['Longitude'] = boatsDF['Country'].apply(get_longitude)

boatsDF.drop('Country', axis=1, inplace=True)

# Save to CSV

In [ ]:
# Reset the indexes
boatsDF = boatsDF.reset_index(drop=True)

# Write to CSV file
boatsDF.to_csv("boatsDFinclNA.csv", sep=";")